<a href="https://colab.research.google.com/github/ayushgopal/Unscripted_Personal/blob/main/Copy_of_transformers_summarization_wandb_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ffmpeg-python


In [ ]:
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
audio = get_audio()



In [ ]:
!pip install pipwin


     |████████████████████████████████| 122kB 6.1MB/s 
     |████████████████████████████████| 1.0MB 34.6MB/s 
  Created wheel for pipwin: filename=pipwin-0.5.1-py2.py3-none-any.whl size=8791 sha256=0aebf476d00e97a73935d285b966450d105239e709f45d67bda4438018a8fd99
  Stored in directory: /root/.cache/pip/wheels/ed/b5/a9/f8bf742bfd2a45bea5b27368a5e3e5a390cda636d40829a1fc
  Created wheel for pyjsparser: filename=pyjsparser-2.7.1-cp37-none-any.whl size=26001 sha256=e3352c91386d747c6a28adc52399594dbaf32bbcc1f2a7fe4305ef3dc58975f1
  Stored in directory: /root/.cache/pip/wheels/a2/73/e6/3e433f3fd78257c3f971baf8cc9001cc0c4797268c61751e89
Successfully built pipwin pyjsparser
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
!pip install PyAudio

  Using cached https://files.pythonhosted.org/packages/ab/42/b4f04721c5c5bfc196ce156b3c768998ef8c0ae3654ed29ea5020c749a6b/PyAudio-0.2.11.tar.gz
  Created wheel for PyAudio: filename=PyAudio-0.2.11-cp37-cp37m-linux_x86_64.whl size=52562 sha256=5bc157992cfdc31d0d790720c7b74c0891b64777942c0ada51188ef6da187ded
  Stored in directory: /root/.cache/pip/wheels/f4/a8/a4/292214166c2917890f85b2f72a8e5f13e1ffa527c4200dcede
Successfully built PyAudio


In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg



Reading package lists... Done
Building dependency tree       
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1).
libportaudiocpp0 is already the newest version (19.6.0-1).
portaudio19-dev is already the newest version (19.6.0-1).
libasound2-dev is already the newest version (1.1.3-5ubuntu0.5).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following packages were automatically installed and are no longer required:
  libpcre2-16-0 libpcre2-32-0 libpcre2-dev libpcre2-posix0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
pip install SpeechRecognition

     |████████████████████████████████| 32.8MB 85kB/s 


In [ ]:
import pyaudio
import speech_recognition as sr

In [ ]:
reco=sr.Recognizer()
audio=get_audio
with get_audio as source: 
   reco.adjust_for_ambient_noise(source) 
   audio = reco.record(source)
   result = reco.recognize_google(audio)

AttributeError: ignored

# Fine Tuning Transformer for Summary Generation

In [1]:
!pip install pytorch_lightning
!pip install transformers==2.9.0
!pip install wandb -q
!pip install sentencepiece
# Code for TPU packages install
# !curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

     |████████████████████████████████| 819kB 26.7MB/s 
     |████████████████████████████████| 829kB 43.3MB/s 
     |████████████████████████████████| 10.6MB 43.4MB/s 
     |████████████████████████████████| 235kB 47.0MB/s 
     |████████████████████████████████| 122kB 49.3MB/s 
     |████████████████████████████████| 645kB 45.7MB/s 
     |████████████████████████████████| 1.3MB 29.8MB/s 
     |████████████████████████████████| 143kB 49.7MB/s 
     |████████████████████████████████| 296kB 53.8MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491070 sha256=cd71ea14be8cb95231e89b6d53c60fcb860a604197c17cea222eefebada381cb
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
ERROR: tensorflow 2.5.0 has requirement tensorboard~=2.5, but you'll have tensorboard 2.4.1 which is incompatible.
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfull

In [2]:
# Importing stock libraries
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

# WandB – Import the wandb library
import wandb

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
# Checking out the GPU we have access to. This is output is from the google colab version. 
!nvidia-smi

Sun Jul 18 10:25:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# # Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

# Preparing for TPU usage
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()

In [5]:
# Login to wandb to log the model run and all the parameters
!wandb login

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
# Creating a custom dataset for reading the dataframe and loading it into the dataloader to pass it to the neural network at a later stage for finetuning the model and to prepare it for predictions

class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.Plot
        self.ctext = self.data.PlotSummary

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [7]:
# Creating the training function. This will be called in the main function. It is run depending on the epoch value.
# The model is put into train mode and then we wnumerate over the training loader and passed to the defined network 

def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, lm_labels=lm_labels)
        loss = outputs[0]
        
        if _%10 == 0:
            wandb.log({"Training Loss": loss.item()})

        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()

In [8]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [9]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping


In [10]:
from pytorch_lightning.loggers import TensorBoardLogger
import pytorch_lightning as pl



In [11]:
def main():
    # WandB – Initialize a new run
    wandb.init(project="chat_summary_2")

      # WandB – Config is a variable that holds and saves hyperparameters and inputs
    # Defining some key variables that will be used later on in the training  
    config = wandb.config          # Initialize config
    config.TRAIN_BATCH_SIZE = 2    # input batch size for training (default: 64)
    config.VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
    config.TRAIN_EPOCHS = 4       # number of epochs to train (default: 10)
    config.VAL_EPOCHS = 1 
    config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
    config.SEED = 42               # random seed (default: 42)
    config.MAX_LEN = 512
    config.SUMMARY_LEN = 150 

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(config.SEED) # pytorch random seed
    np.random.seed(config.SEED) # numpy random seed
    torch.backends.cudnn.deterministic = True

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained("t5-base")
    

    # Importing and Pre-Processing the domain data
    # Selecting the needed columns only. 
    # Adding the summarzie text in front of the text. This is to format the dataset similar to how T5 model was trained for summarization task. 
    df = pd.read_csv('final_chat.csv',encoding='latin-1')
    df = df[['dialogue','summary']]
    df.columns=['Plot','PlotSummary']
    df.PlotSummary = 'summarize: ' + df.PlotSummary
    print(df.head())

    
    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest will be used for validation. 
    train_size = 0.8
    train_dataset=df.sample(frac=train_size,random_state = config.SEED)
    val_dataset=df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    print("FULL Dataset: {}".format(df.shape))
    print("TRAIN Dataset: {}".format(train_dataset.shape))
    print("TEST Dataset: {}".format(val_dataset.shape))


    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
    val_set = CustomDataset(val_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

    # Defining the parameters for creation of dataloaders
    train_params = {
        'batch_size': config.TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

    val_params = {
        'batch_size': config.VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 0
        }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)


    
    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained("t5-base")
    model = model.to(device)

    # Defining the optimizer that will be used to tune the weights of the network in the training session. 
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)
    %load_ext tensorboard
    %tensorboard --logdir ./lightning_logs

    checkpoint_callback= ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
    )
    logger= TensorBoardLogger("lightning_logs", name="chat_summary_2")

    trainer= pl.Trainer(
    logger=logger,
    checkpoint_callback=checkpoint_callback,
    max_epochs=8,
    gpus=1,
    progress_bar_refresh_rate=30,

   )
    
    
    # Log metrics with wandb
    wandb.watch(model, log="all")
    # Training loop
    print('Initiating Fine-Tuning for the model on our dataset')

    for epoch in range(config.TRAIN_EPOCHS):
        train(epoch, tokenizer, model, device, training_loader, optimizer)


    # Validation loop and saving the resulting file with predictions and acutals in a dataframe.
    # Saving the dataframe as predictions.csv
    print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe')
    for epoch in range(config.VAL_EPOCHS):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
        final_df.to_csv('blanknew.csv')
        print('Output Files generated for review')

if __name__ == '__main__':
    main()

wandb: Currently logged in as: ayush1 (use `wandb login --relogin` to force relogin)



                                                Plot                                        PlotSummary
0  Amanda: I baked  cookies. Do you want some?\r\...  summarize: Amanda baked cookies and will bring...
1  Olivia: Who are you voting for in this electio...  summarize: Olivia and Olivier are voting for l...
2  Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...  summarize: Kim may try the pomodoro technique ...
3  Edward: Rachel, I think I'm in ove with Bella....  summarize: Edward thinks he is in love with Be...
4  Sam: hey  overheard rick say something\r\nSam:...  summarize: Sam is confused, because he overhea...
FULL Dataset: (14732, 2)
TRAIN Dataset: (11786, 2)
TEST Dataset: (2946, 2)


ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2021-07-18 10:28:11.935856: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/main.py", line 65, in run_main
    default.get_plugins(),
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/default.py", line 108, in get_plugins
    return get_static_plugins() + get_dynamic_plugins()
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/default.py", line 146, in get_dynamic_plugins
    "tensorboard_plugins"
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/default.py", 

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


Initiating Fine-Tuning for the model on our dataset
Epoch: 0, Loss:  7.533932685852051
Epoch: 0, Loss:  2.666146755218506
Epoch: 0, Loss:  2.932476282119751
Epoch: 0, Loss:  2.851024866104126
Epoch: 0, Loss:  3.5675384998321533
Epoch: 0, Loss:  3.0714728832244873
Epoch: 0, Loss:  2.5347821712493896
Epoch: 0, Loss:  3.0767385959625244
Epoch: 0, Loss:  2.5606157779693604
Epoch: 0, Loss:  2.844151496887207
Epoch: 0, Loss:  2.86934232711792
Epoch: 0, Loss:  2.1976096630096436
Epoch: 1, Loss:  2.9946048259735107
Epoch: 1, Loss:  2.219844102859497
Epoch: 1, Loss:  2.666886806488037
Epoch: 1, Loss:  2.3879356384277344
Epoch: 1, Loss:  2.8661370277404785
Epoch: 1, Loss:  2.892265796661377
Epoch: 1, Loss:  2.326533317565918
Epoch: 1, Loss:  2.2087442874908447
Epoch: 1, Loss:  2.2195301055908203
Epoch: 1, Loss:  2.635563373565674
Epoch: 1, Loss:  2.2207844257354736
Epoch: 1, Loss:  2.4921720027923584
Epoch: 2, Loss:  2.162449598312378
Epoch: 2, Loss:  2.008103609085083
Epoch: 2, Loss:  2.3269662

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


Completed 0
Completed 100
Completed 200
Completed 300
Completed 400
Completed 500
Completed 600
Completed 700
Completed 800
Completed 900
Completed 1000
Completed 1100
Completed 1200
Completed 1300
Completed 1400
Output Files generated for review


In [22]:
final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
final_df.to_csv('blanknew.csv')
print('Output Files generated for review')


NameError: ignored

In [21]:
import pickle
filename = 'finalized_chat_model.pkl'
pickle.dump(model, open(filename, 'wb'))

NameError: ignored

In [14]:
model

NameError: ignored

In [ ]:
mo

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras


In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")

In [ ]:
checkpoint_path = "checkpoints"
checkpoint_dir = os.path.dirname(checkpoint_path)
wandb.init(project="movie_summarization")

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
wandb.watch(model, log="all")

[]

In [ ]:
def summarize(text):
  text_encoding = tokenizer(
      text,
      max_length=512,
      padding="max_length",
      truncation=True,
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"

  )
      
  generation_ids=trained_model.model.generate(
      input_ids=text_encoding["input_ids"],
      attention_mask=text_encoding["attention_mask"],
      max_length=150,
      num_beams=2,
      repetation_penalty=2.5,
      length_penalty=1.0,
      early_stopping=True
  )
preds=[
       tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
       for gen_id in generation_ids
]
return "".join(preds)


NameError: ignored

In [20]:
config = wandb.config          # Initialize config
config.TRAIN_BATCH_SIZE = 2    # input batch size for training (default: 64)
config.VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
config.TRAIN_EPOCHS = 2        # number of epochs to train (default: 10)
config.VAL_EPOCHS = 1 
config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
config.SEED = 42               # random seed (default: 42)
config.MAX_LEN = 512
config.SUMMARY_LEN = 150 

torch.manual_seed(config.SEED) # pytorch random seed
np.random.seed(config.SEED) # numpy random seed
torch.backends.cudnn.deterministic = True

    # tokenzier for encoding the text
tokenizer = T5Tokenizer.from_pretrained("t5-base")
    



    
    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest will be used for validation. 



    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.



    
    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
    # Further this model is sent to device (GPU/TPU) for using the hardware.


    





    # Validation loop and saving the resulting file with predictions and acutals in a dataframe.
    # Saving the dataframe as predictions.csv
print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe')
val_loader="hello my name is AAAYUSJDJ and I am from iit bombay"
predictions, actuals = validate(1, tokenizer, model, device, val_loader)
print(predictions)




Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe


NameError: ignored

In [ ]:
df=df.dropna

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("best-checkpoint")
model = AutoModelWithLMHead.from_pretrained("best-checkpoint")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

text = """"
Lack of fluids can lead to dry mouth, which is a leading cause of bad breath. Water
can also dilute any chemicals in your mouth or gut that are causing bad breath., Studies show that
eating 6 ounces of yogurt a day reduces the level of odor-causing compounds in the mouth. In
particular, look for yogurt containing the active bacteria Streptococcus thermophilus or
Lactobacillus bulgaricus., The abrasive nature of fibrous fruits and vegetables helps to clean
teeth, while the vitamins, antioxidants, and acids they contain improve dental health.Foods that can
be particularly helpful include:Apples — Apples contain vitamin C, which is necessary for health
gums, as well as malic acid, which helps to whiten teeth.Carrots — Carrots are rich in vitamin A,
which strengthens tooth enamel.Celery — Chewing celery produces a lot of saliva, which helps to
neutralize bacteria that cause bad breath.Pineapples — Pineapples contain bromelain, an enzyme that
cleans the mouth., These teas have been shown to kill the bacteria that cause bad breath and
plaque., An upset stomach can lead to burping, which contributes to bad breath. Don’t eat foods that
upset your stomach, or if you do, use antacids. If you are lactose intolerant, try lactase tablets.,
They can all cause bad breath. If you do eat them, bring sugar-free gum or a toothbrush and
toothpaste to freshen your mouth afterwards., Diets low in carbohydrates lead to ketosis — a state
in which the body burns primarily fat instead of carbohydrates for energy. This may be good for your
waistline, but it also produces chemicals called ketones, which contribute to bad breath.To stop the
problem, you must change your diet. Or, you can combat the smell in one of these ways:Drink lots of
water to dilute the ketones.Chew sugarless gum or suck on sugarless mints.Chew mint leaves.
"""

preprocess_text = text.strip().replace("\n","")
tokenized_text = tokenizer.encode(preprocess_text, return_tensors="pt").to(device)

summary_ids = model.generate(
            tokenized_text,
            max_length=150, 
            num_beams=2,
            repetition_penalty=2.5, 
            length_penalty=1.0, 
            early_stopping=True
        )

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nSummarized text: \n",output)

OSError: ignored

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# initialize the model architecture and weights
model = T5ForConditionalGeneration.from_pretrained("t5-base")
# initialize the model tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")

In [ ]:
article = """

Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric: I know! And shows how Americans see Russian ;)\r\nRob: And it's really funny!\r\nEric: I know! I especially like the train part!\r\nRob: Hahaha! No one talks to the machine like that!\r\nEric: Is this his only stand-up?\r\nRob: Idk. I'll check.\r\nEric: Sure.\r\nRob: Turns out no! There are some of his stand-ups on youtube.\r\nEric: Gr8! I'll watch them now!\r\nRob: Me too!\r\nEric: MACHINE!\r\nRob: MACHINE!\r\nEric: TTYL?\r\nRob: Sure :)
"""

In [ ]:
# encode the text into tensor of integers using the appropriate tokenizer
inputs = tokenizer.encode("summarize: " + article, return_tensors="pt", max_length=512, truncation=True)

In [ ]:

# generate the summarization output
outputs = model.generate(
            inputs, 
            max_length=150, 
            num_beams=2,
            repetition_penalty=2.5, 
            length_penalty=1.0, 
            early_stopping=True)
# just for debugging
print(outputs)
print(tokenizer.decode(outputs[0]))

tensor([[    0,   192,  6562,   336,   131,     3,  4241,  3978,     6,  5608,
            53,    13,    80,  2538,    16,     3,     9,  1679,   383,  2265,
             3,     5,     8,   166,    19,   356,    16,     3,     7, 28478,
         12215,    26,    23,   302,    11,  1217,   286,    16,     3,     9,
           315,   294,    30,     3,    17,    76,    15,    18,  1135,  2071,
            12,     8,  1861,    45,     8,   511,     3, 25403,    16,     3,
             9,  3300,   315,   294,    13,     8,  1679,     3,    88, 14432,
            30,   112,  4952,     3,   233]])
two shots last just 61 seconds, consisting of one shot in a wood during winter. the first is set in southeastern floridius and takes place in a different part on tue-day path to the camera from the second filmed in a slightly different part of the wood he rides on his horse...


In [ ]:
!transformers-cli login


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
2021-07-17 13:27:57.956245: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: ayushmgp1234@gmail.com
Password: 
Login successful


In [ ]:
tokenizer.push_to_hub("trail")


AttributeError: ignored

In [ ]:
for epoch in range(config.VAL_EPOCHS):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
        final_df.to_csv('blank.csv')
        print('Output Files generated for review')

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fb2e160e310>> (for pre_run_cell):


Exception: ignored

Completed 0


KeyboardInterrupt: ignored

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fb2e160e310>> (for post_run_cell):


Exception: ignored

In [ ]:
final_df

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fb2e160e310>> (for pre_run_cell):


Exception: ignored

,Generated Text,Actual Text
0,<extra_id_0>. Eric and Rob are going to watch ...,Eric: MACHINE!rnRob: That's so gr8!rnEric: I k...
1,<extra_id_0> Ollie and Jane have a party. Olli...,"Ollie: Hi, are you in WarsawrnJane: yes, just ..."
2,"<extra_id_0> Tom.<extra_id_1>,<extra_id_2> a c...",Mary: Are you going by car or train?nTom: Ella...
3,<extra_id_0> Luke will send Luke his bank deta...,Luke: are you still looking for someone to joi...
4,<extra_id_0> Deidre offers her a few hours on ...,"Deirdre: Hi Beth, how are you love?rnBeth: Hi ..."
...,...,...
131,"<extra_id_0> the rent money, because John sent...",Kelly: I still haven't received the rent money...
132,James is James's James has the game. James and...,Hannah: Are you watching the game?rnJames: No ...
133,<extra_id_0>. Autumn has sewn a dress.<extra_i...,Autumn: u sewn anything?rnAutumn: show me show...
134,Sean never gets up for Sean not attends the mo...,Mike: who's going to the lecture tomorrow morn...


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fb2e160e310>> (for post_run_cell):


Exception: ignored